In [1]:
import sys
sys.path.append("..")

from shared.GoogleServices.GoogleDriveService import GoogleDriveService

In [2]:
from pathlib import Path
import json
from google.oauth2.credentials import Credentials

credentials_info = json.load(Path("../google_token/logged_in_credentials.json").open())
credentials = Credentials.from_authorized_user_info(credentials_info)

In [3]:
wrapper = GoogleDriveService(credentials)
wrapper.authenticate()

In [4]:
top_folder_id = "19gdVV_DMtdQU0xi7TgfKJCRRc4c7m0fd"

In [5]:
from io import BytesIO

with open("testi_attachment.pdf", "rb") as file:
    pdf_bytes = BytesIO(file.read())

In [6]:
wrapper.upload_file_directly(pdf_bytes, "testi_attachment.pdf", top_folder_id)

'1FyTEbPH2s2soN0MZPIkDKpox9jTf2Q0n'

In [ ]:
def find_or_create_folder(service, folder_name: str, parent_id: str) -> str:
    """
    Given a folder name and a parent folder ID, check if a folder
    with that name already exists under the parent. If it does not,
    create it. Return the folder's ID.
    """
    # 1. Search for an existing folder with this name under parent_id
    query = (
        f"mimeType = 'application/vnd.google-apps.folder' "
        f"and name = '{folder_name}' "
        f"and '{parent_id}' in parents "
        f"and trashed = false"
    )
    
    results = service.files().list(
        q=query,
        spaces='drive',
        fields='files(id, name)',
    ).execute()
    
    folders = results.get('files', [])
    if len(folders) > 0:
        # Folder already exists; just return its ID
        return folders[0]['id']
    
    # 2. Create the folder if not found
    folder_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_id]
    }
    folder = service.files().create(
        body=folder_metadata,
        fields='id'
    ).execute()
    
    return folder.get('id')


In [11]:
testi_id = find_or_create_folder(wrapper.service, "testi", top_folder_id)
testi_id

'1NE6K4IChAQd5XlLMq5hz2BIxUmba2i0V'

In [10]:
def get_folder_id_by_path(service, root_folder_id: str, path: list[str]) -> str:
    """
    Given a path like ['sub1', 'sub2', 'sub3'] (meaning sub1/sub2/sub3), walk through each subfolder
    under root_folder_id. For each subfolder, find or create it.
    Return the ID of the final subfolder.
    """
    current_parent_id = root_folder_id
    # Iterate through each folder name in the provided path
    for folder_name in path:
        current_parent_id = find_or_create_folder(service, folder_name, current_parent_id)
    return current_parent_id


In [12]:
get_folder_id_by_path(wrapper.service, testi_id, ["sub1", "sub2", "sub3"])

'1ggZztnffRcHmsa-iE4X3WLWksKTXBny3'

In [13]:
get_folder_id_by_path(wrapper.service, testi_id, ["sub1", "sub2"])

'1GBVapr5ntarqsJ-1X_HrpLh7xxfRZ-Gw'